In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os

# Laden des DataFrames und Setzen der ersten Spalte nicht als Index
df = pd.read_json(r"C:/Users/gueld/Desktop/Portfolio Projekt/Daten/github/steamdb.json")

# Überprüfen auf fehlende Werte in der Spalte 'full_price' (full_price und current_price haben gleich viele missing values)
missing_full_price = df[df['full_price'].isnull()]

# Ausgeben der Einträge der Spalte 'store_url' bei fehlenden Werten in 'full_price'
missing_urls = missing_full_price['store_url']

# Funktion zum Laden der URLs aus der Datenbankdatei mit fehlenden Werten in 'full_price' und 'current_price'
def load_missing_urls(database_path):
    df = pd.read_json(database_path)
    missing_full_price = df[df['full_price'].isnull() & df['current_price'].isnull()]
    missing_urls = missing_full_price['store_url'].tolist()
    return missing_urls

# Funktion zum Scrapen einer Steam-Seite
def scrape_steam_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        data = {}
        
        title_element = soup.find('div', class_='apphub_AppName')
        data['name'] = title_element.get_text() if title_element else 'N/A'
        
        current_price_element = soup.find('div', class_='game_purchase_price')
        data['current_price'] = current_price_element.get_text().strip() if current_price_element else 'N/A'
        
        full_price_element = soup.find('div', class_='discount_original_price')
        data['full_price'] = full_price_element.get_text().strip() if full_price_element else 'N/A'
        
        data['store_url'] = url
        
        return data
    else:
        print(f"Fehler beim Laden der Seite: {response.status_code}")
        return None

# Funktion zum Speichern von Daten in einer JSON-Datei
def save_to_json(data, file_index, output_dir):
    file_path = os.path.join(output_dir, f'steam_data_{file_index}.json')
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Gespeichert: {file_path}")

# Pfade und Einstellungen
database_path = r"C:/Users/gueld/Desktop/Portfolio Projekt/Daten/github/steamdb.json"
output_dir = r"C:/Users/gueld/Desktop/Portfolio Projekt/Daten/steam_scrape_json/neu/steam_price.json"
os.makedirs(output_dir, exist_ok=True)

# Lade die URLs aus der Datenbank mit fehlenden Werten in 'full_price' und 'current_price'
urls = load_missing_urls(database_path)

# Initialisiere Variablen - speichere jeweils 500 zeilen in einer .json Datei
scraped_data = []
file_index = 1
batch_size = 500

# Iteriere über die URLs und scrape jede Seite
for url in urls:
    print(f"Scraping URL: {url}")
    data = scrape_steam_page(url)
    if data:
        scraped_data.append(data)
    
    # Speichere die Daten in einer JSON-Datei, wenn die Batch-Größe erreicht ist
    if len(scraped_data) >= batch_size:
        save_to_json(scraped_data, file_index, output_dir)
        scraped_data = []  # Leere die Liste für den nächsten Batch
        file_index += 1

# Speichere verbleibende Daten in einer letzten JSON-Datei
if scraped_data:
    save_to_json(scraped_data, file_index, output_dir)